In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

import matplotlib
%matplotlib inline

matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['figure.figsize'] = (14,9)
matplotlib.rcParams['savefig.bbox'] = 'tight'

In [65]:
import importlib
importlib.reload(resnet)

<module 'resnet' from '/Users/albertxu/AI4AM/models/resnet.py'>

In [6]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import pickle
import resnet

import taichiUtil

In [14]:
batch_size = 10
vals_ds = tfds.load('taichiSim', split=[
    tfds.core.ReadInstruction('tmp', from_=k, to=k+10, unit='%')
    for k in range(0, 100, 10)], batch_size=batch_size)
trains_ds = tfds.load('taichiSim', split=[
    (tfds.core.ReadInstruction('tmp', to=k, unit='%') +
     tfds.core.ReadInstruction('tmp', from_=k+10, unit='%'))
    for k in range(0, 100, 10)], batch_size=batch_size)

In [81]:
model = resnet.resnet_18(drop_p=0, n_class=2)
objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001)

In [82]:
def loss(m, x, y, _train=True):
    _y = m(x, training=_train)
    return objective(y_true=y, y_pred=_y)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, _train=True)
        return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [83]:
train_losses = []
train_accs = []
test_losses = []
test_accs = []

In [84]:
train = trains_ds[0]
val = vals_ds[0]

In [ ]:
# Training loop
n_epoch = 500
for epoch in range(n_epoch):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    test_epoch_loss_avg = tf.keras.metrics.Mean()
    test_epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    train2 = train.shuffle(buffer_size=50)
    for ex in train2:
        x = tf.image.convert_image_dtype(ex['image'], tf.float32)
        ys = taichiUtil.dat2vec(ex)
        # class_encode = tf.reshape(tf.one_hot((ys[:,3:7] > 0).astype(int), 2), (-1, 8))
        y = (ys[:,3] > 0).astype(int)
        l, grads = grad(model, x, y)

        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss_avg.update_state(l)
        epoch_accuracy.update_state(y, model(x, training=True))
    
    print(f'Epoch {epoch:03d}, Train Loss: {epoch_loss_avg.result():.3f}, Acc: {epoch_accuracy.result():.3f}')


13